In [2]:
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, LineString, MultiPolygon
from shapely.ops import split
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import copy
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
burned = pd.read_csv("Data Used/burnedsampleData.csv")
unburned = pd.read_csv("Data Used/unburnedsampleData.csv")

In [6]:
burned.head()

,FIRE_NAME,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,burnSeverity,dNBR,...,rangeland_bare_ground,rangeland_big_sagebrush,rangeland_herbaceous,rangeland_litter,rangeland_sagebrush,rangeland_sagebrush_height,rangeland_shrub,rangeland_shrub_height,x_coord,y_coord
0,ATLAS,7901.0,8141,8855,9145,13120,12799,10556,1,9.171320,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-185765.625002,36129.374898
1,ATLAS,7936.0,8119,8861,9040,13384,11922,9801,1,11.142816,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-185776.875002,35589.374898
2,ATLAS,8341.0,8542,9265,9723,17122,13685,11178,1,14.236607,...,6.0,0.0,53.0,7.0,0.0,0.0,27.0,48.0,-185776.875002,35994.374898
3,ATLAS,8833.0,9513,11150,12624,17102,16821,12897,1,-13.036240,...,7.0,0.0,44.0,7.0,0.0,0.0,40.0,60.0,-185788.125002,36354.374898
4,ATLAS,8048.0,8233,8796,8842,13675,12234,10061,1,70.785240,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-185799.375002,37546.874898


In [7]:
temp = burned.loc[:, ~burned.columns.isin(['SR_B5','SR_B7','SR_B4','geometry','FIRE_NAME','impervious_descriptor'])]
utemp = unburned.loc[:, ~unburned.columns.isin(['SR_B5','SR_B7','SR_B4','geometry','FIRE_NAME','impervious_descriptor'])]
for i in temp.columns:
    burned[i]=pd.to_numeric(temp[i], errors='coerce')
    unburned[i]=pd.to_numeric(utemp[i], errors='coerce')
df = pd.concat([unburned,burned])
df

,FIRE_NAME,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,burnSeverity,dNBR,...,rangeland_bare_ground,rangeland_big_sagebrush,rangeland_herbaceous,rangeland_litter,rangeland_sagebrush,rangeland_sagebrush_height,rangeland_shrub,rangeland_shrub_height,x_coord,y_coord
0,ATLAS,7273.0,7451.0,7730,7416,7395,7397,7348,1,0.107050,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-183748.125002,57165.624898
1,ATLAS,9116.0,9973.0,11849,14092,19003,18687,14163,1,-24.181034,...,24.0,0.0,52.0,11.0,0.0,0.0,6.0,15.0,-183775.625002,56436.874898
2,ATLAS,7744.0,7961.0,8575,8730,13556,11451,9479,1,-6.647168,...,6.0,0.0,8.0,24.0,0.0,0.0,59.0,158.0,-183775.625002,56904.374898
3,ATLAS,7368.0,7456.0,7593,7511,8461,7934,7579,1,99.826752,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-183803.125002,54484.374898
4,ATLAS,7760.0,7843.0,8141,8340,11399,9788,8657,1,27.707573,...,4.0,0.0,25.0,10.0,0.0,0.0,41.0,141.0,-183803.125002,55804.374898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149223,WALKER,7457.0,7514.0,7745,7602,9824,8140,7737,2,107.069328,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-59833.125001,225343.124897
149224,WALKER,7497.0,7555.0,7873,7689,10645,8841,8075,2,115.286041,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-59979.375001,225320.624897
149225,WALKER,7514.0,7567.0,7967,7739,11917,9070,8154,1,68.903618,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-60181.875001,225331.874897
149226,WALKER,7457.0,7534.0,7892,7712,10404,8601,7972,1,53.100204,...,101.0,101.0,101.0,101.0,101.0,998.0,101.0,998.0,-60328.125001,225455.624897


In [8]:
clean_df = df.loc[:, ~df.columns.isin(['NBR','dNBR','SR_B5','SR_B7','SR_B4','geometry','FIRE_NAME','impervious_descriptor'])]
for i in clean_df.columns:
    clean_df[i].fillna(value=clean_df[i].mean(), inplace=True)
X_train = clean_df.drop(['burnSeverity'],axis=1)
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_scaled = scaler.transform(X_train)
X_scaled

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


array([[-1.05177242, -0.98321993, -1.06533515, ...,  1.14035354,
        -0.06434863, -1.31206228],
       [ 1.20994343,  1.71555454,  2.42421759, ..., -1.47661979,
        -0.06466261, -1.31740393],
       [-0.47376474, -0.43747251, -0.3494643 , ..., -1.09592072,
        -0.06466261, -1.31397721],
       ...,
       [-0.75601902, -0.85908914, -0.86455244, ...,  1.14035354,
         1.34647285, -0.0794256 ],
       [-0.825969  , -0.89440221, -0.92809127, ...,  1.14035354,
         1.34480304, -0.07851853],
       [-0.63207257, -0.72425743, -0.67224157, ...,  1.14035354,
         1.34428925, -0.07950806]])

In [13]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = LinearRegression()
pipe.fit(x_train, y_train)
print("Linear Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Linear Regression: 0.5324782240486086
SR_B1: 0.4164296268759279
SR_B2: -0.10958956980285112
SR_B3: -0.5904353823539316
SR_B6: 0.36614264344740244
NDVI: -0.47607364960306725
elevation: 0.0657093034914243
pr: -0.003418389607113671
rmax: -0.16512432373080782
rmin: -0.01563072040484141
sph: -0.2634508115482871
srad: 0.3532159456441533
th: 0.012179113646699935
tmmn: 0.17117731589958204
tmmx: 0.36882404679882685
vs: 0.059556073295658964
erc: -0.4486713221355194
eto: -1.1458985086333195
bi: 0.022453586735275238
fm100: 0.0773266938034661
fm1000: -0.1486950972125967
etr: 0.8098990690911755
vpd: -0.2966297071254419
landcover: 0.0927571739395212
impervious: 0.019249660135982408
percent_tree_cover: 0.27213968146423534
rangeland_annual_herbaceous: -0.20020936974015496
rangeland_bare_ground: -0.19022148973053057
rangeland_big_sagebrush: -0.4807305307384926
rangeland_herbaceous: -0.01761251235648108
rangeland_litter: -0.13175377543461303
rangeland_sagebrush: -1.2875951848772949
rangeland_sagebrush_he

['SR_B1',
 'SR_B6',
 'elevation',
 'srad',
 'tmmn',
 'tmmx',
 'vs',
 'bi',
 'fm100',
 'etr',
 'landcover',
 'percent_tree_cover',
 'rangeland_sagebrush_height',
 'rangeland_shrub',
 'x_coord']

In [14]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = LogisticRegression()
pipe.fit(x_train, y_train)
print("Logistic Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_[0]
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Logistic Regression: 0.7905119475643514
SR_B1: -1.631696414773089
SR_B2: 0.33079549188924556
SR_B3: 5.103393473790319
SR_B6: -4.175730681979149
NDVI: 4.472413567871374
elevation: 0.4297614499803496
pr: -0.05630296466976176
rmax: 0.23384161848475915
rmin: 1.0673180831814106
sph: 0.8098605536800759
srad: -0.9377273852643977
th: -0.05662159696734318
tmmn: -0.8949737472727265
tmmx: -0.15052447458989301
vs: -0.7955315944877234
erc: 0.21245218284727363
eto: 0.06674026721797235
bi: 0.6487586443123139
fm100: -0.8106019637124758
fm1000: 0.6199426727172842
etr: 0.09686230828514032
vpd: -0.21960693100512185
landcover: -0.327988905182512
impervious: -0.3084541374494184
percent_tree_cover: -2.7699281073912987
rangeland_annual_herbaceous: 0.754188479609005
rangeland_bare_ground: 0.9923408612532589
rangeland_big_sagebrush: -0.313362762068762
rangeland_herbaceous: 0.5392714229516151
rangeland_litter: -0.011970939108475045
rangeland_sagebrush: -0.4183508607623895
rangeland_sagebrush_height: -0.23501608

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['SR_B2',
 'SR_B3',
 'NDVI',
 'elevation',
 'rmax',
 'rmin',
 'sph',
 'erc',
 'eto',
 'bi',
 'fm1000',
 'etr',
 'rangeland_annual_herbaceous',
 'rangeland_bare_ground',
 'rangeland_herbaceous',
 'x_coord',
 'y_coord']

In [9]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(clean_df[['burnSeverity']].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = RandomForestRegressor()
pipe.fit(x_train, y_train)
pipe.score(x_test, y_test)  

/Users/alicelu/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8045643104426572

In [10]:
importance = pipe.feature_importances_
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

SR_B1: 0.01763677856039855
SR_B2: 0.018628442036516764
SR_B3: 0.04751461616440036
SR_B6: 0.07415553644496928
NDVI: 0.36599546444644837
elevation: 0.02462676994301247
pr: 0.005520979476220605
rmax: 0.008238556743515474
rmin: 0.00579961541904869
sph: 0.013300789953006816
srad: 0.011485192804840083
th: 0.008355747806559425
tmmn: 0.006278492578684149
tmmx: 0.014414470334893912
vs: 0.011629933686045597
erc: 0.003691295521549333
eto: 0.005074078876418118
bi: 0.005967099165232974
fm100: 0.004392826070421919
fm1000: 0.00558246381089168
etr: 0.0046155996672062665
vpd: 0.004688072483418369
landcover: 0.023722427546842458
impervious: 0.001272256549098544
percent_tree_cover: 0.16205943683260673
rangeland_annual_herbaceous: 0.003521327636769791
rangeland_bare_ground: 0.007766678996280859
rangeland_big_sagebrush: 0.0005769927264267481
rangeland_herbaceous: 0.0040830098128168
rangeland_litter: 0.0067051152416384785
rangeland_sagebrush: 0.0007462878683054101
rangeland_sagebrush_height: 0.0008175164952

['SR_B3',
 'SR_B6',
 'NDVI',
 'elevation',
 'landcover',
 'percent_tree_cover',
 'x_coord',
 'y_coord']